## TEEHR Post-Event Example 2
### Explore Observed and Forecast Timeseries from a Recent Flood Event

Add more text description about this use case....





### Install and Import packages

In [ ]:
%%capture
#!pip install 'teehr @ git+https://@github.com/RTIInternational/teehr@main'

In [ ]:
import teehr.queries.duckdb as tqd

import postevent_dashboard_utils as du
import importlib
from datetime import datetime, timedelta
from pathlib import Path
import pandas as pd
import geopandas as gpd
import holoviews as hv
from holoviews.element import tiles
import geoviews as gv
import panel as pn
hv.extension('bokeh', logo=False)

In [ ]:
# evaluation study directory
STUDY_DIR = Path("/home", "jovyan", "shared", "rti-eval", "post-event-example")

## general units ('english' or 'metric') to show in visualization
viz_units = "metric"

# evaluation scenario definitions - specific variables and configurations to be compared within the overall study

# medium range streamflow forecast evaluation files 
MRF_streamflow = dict(
    scenario_name="medium_range",
    variable="streamflow",
    primary_filepath=Path(STUDY_DIR, "timeseries", "usgs", "*.parquet"),
    secondary_filepath=Path(STUDY_DIR, "timeseries", "medium_range_mem1", "*.parquet"),
    crosswalk_filepath=Path(STUDY_DIR, "geo", "usgs_nwm22_crosswalk.parquet"),
    geometry_filepath=Path(STUDY_DIR, "geo", "usgs_geometry.parquet")
)

# medium range precip forecast evaluation files
MRF_forcing = dict(
    scenario_name="medium_range",
    variable="precipitation",    
    primary_filepath=Path(STUDY_DIR, "timeseries", "forcing_analysis_assim", "*.parquet"),
    secondary_filepath=Path(STUDY_DIR, "timeseries", "forcing_medium_range", "*.parquet"),
    crosswalk_filepath=Path(STUDY_DIR, "geo", "huc10_huc10_crosswalk.parquet"),                    # the primary and secondary are both HUC10
    geometry_filepath=Path(STUDY_DIR, "geo", "huc10_geometry.parquet"),
)

# short range streamflow forecast evaluation files 
SRF_streamflow = dict(
    scenario_name="short_range",
    variable="streamflow",
    primary_filepath=MRF_streamflow["primary_filepath"],
    secondary_filepath=Path(STUDY_DIR, "timeseries", "short_range", "*.parquet"),
    crosswalk_filepath=MRF_streamflow["crosswalk_filepath"],
    geometry_filepath=MRF_streamflow["geometry_filepath"],
)

# medium range precip forecast evaluation files
SRF_forcing = dict(
    scenario_name="short_range",
    variable="precipitation",    
    primary_filepath=MRF_forcing["primary_filepath"],
    secondary_filepath=Path(STUDY_DIR, "timeseries", "forcing_short_range", "*.parquet"),
    crosswalk_filepath=MRF_forcing["crosswalk_filepath"],
    geometry_filepath=MRF_forcing["geometry_filepath"],
)

scenario_definitions = [MRF_streamflow, MRF_forcing, SRF_streamflow, SRF_forcing]

attribute_paths = dict(
    usgs_upstream_area=Path(STUDY_DIR, "geo", "usgs_attr_upstream_area.parquet"),
    usgs_ecoregions=Path(STUDY_DIR, "geo", "usgs_attr_ecoregions.parquet"),
    usgs_stream_order=Path(STUDY_DIR, "geo", "usgs_attr_stream_order.parquet"),
    usgs_huc_crosswalk=Path(STUDY_DIR, "geo", "usgs_huc12_crosswalk.parquet"),
)
attribute_df = du.combine_attributes(attribute_paths,viz_units)

gage_basins_filepath = Path("/home/jovyan/temp/data/gage_basins.parquet")
gage_basins_gdf = gpd.read_parquet(gage_basins_filepath).to_crs("EPSG:3857")

## Select the scenario and date ranges before launching the dashboard

Next we will check the dates available in the parquet files, and use a slider to select all or a portion of the total available period to evaluate.
(ToDo: create utility to check that data are complete for all of the above defined timeseries files between the min/max dates).

In [ ]:
importlib.reload(du)
#huc2_selector = du.get_huc2_selector()
scenario_selector = du.get_scenario_selector(scenario_name_list=sorted(du.get_scenario_names(scenario_definitions))) 
value_time_slider = du.get_value_time_slider(scenario_definitions)
pn.Column(pn.Spacer(height=10),
          pn.Row(scenario_selector, pn.Spacer(width=20), value_time_slider, width = 1000))

## Explore forecast data, one forecast at a time

In [ ]:
importlib.reload(du)
include_metrics=['max_perc_diff','max_time_diff']
layout = du.post_event_dashboard_2(
    scenario_definitions,
    scenario_selector,
    #huc2_selector,
    value_time_slider,
    attribute_paths,
    include_metrics,
    viz_units,
    gage_basins_gdf,
)
layout

### Below code moved into postevent_dashboard_utils.py to run as function (above), below kept for now for easier troubleshooting if needed, will remove before workshop

In [ ]:
importlib.reload(du)

######### Build components for the dashboard
include_metrics=['max_perc_diff','max_time_diff']

huc2_selector = du.get_huc2_selector()
metric_selector = du.get_single_metric_selector(
    metrics=du.get_metric_selector_dict(metrics=include_metrics))
scenarios = du.get_scenario(scenario_definitions, scenario_name=scenario_selector.value)
scenario_text = du.get_scenario_text(scenario_selector.value)
streamflow_scenario = [s for s in scenarios if s['variable']=='streamflow'][0]
precip_scenario = [s for s in scenarios if s['variable']=='precipitation'][0]

# reference time player (eventually replace with individual arrows)
reference_time_player = du.get_reference_time_player_selected_dates(
    scenario=scenarios, 
    start=value_time_slider[1].value_start-timedelta(hours=1), 
    end=value_time_slider[1].value_end
)
current_ref_time = pn.bind(du.get_reference_time_text, reference_time=reference_time_player.param.value)

# Some universal plot settings
map_opts = dict(show_grid=False, show_legend=False, xaxis = None, yaxis = None, width=600, height=500)
ts_opts = dict(toolbar = None, tools=["hover"], show_title = False)

# Build background map Elements
tiles_background = gv.tile_sources.CartoLight #OSM
timeseries_legend = du.get_separate_legend()

# link widgets to query and build points element
points_bind = pn.bind(
    du.build_points_from_query,
    scenario = streamflow_scenario,
    huc_id=huc2_selector.param.value,
    value_time_start=value_time_slider[1].param.value_start,
    value_time_end=value_time_slider[1].param.value_end,
    reference_time_single=reference_time_player.param.value,
    #value_min=0,   
    group_by=['primary_location_id','reference_time'],    
    include_metrics=include_metrics,#['primary_maximum','max_value_delta','max_value_timedelta'],    
    #metric_limits=dict(primary_maximum=(0.1, 10e6)),
    plot_metric=metric_selector.param.value,
    attribute_paths=attribute_paths,
    units=viz_units,
)
points_dmap = hv.DynamicMap(points_bind)

# Define stream source as points selection from points_dmap
point_selection = hv.streams.Selection1D(source=points_dmap)#, index=[0])

gage_basin_bind = pn.bind(
    du.get_gage_basin_selected_point,
    index=point_selection.param.index,
    points_dmap = points_dmap,  
    gage_basins_gdf = gage_basins_gdf,
    )
gage_basin_dmap = hv.DynamicMap(gage_basin_bind)
#gage_basin_dmap.opts(line_color='k', line_width=2, fill_alpha=0)

# link selected point to query and build timeseries element
hyetograph_bind = pn.bind(
    du.build_hyetograph_from_query_selected_point,
    index=point_selection.param.index,
    points_dmap = points_dmap,          
    scenario = precip_scenario,
    reference_time_single=reference_time_player.param.value,
    value_min=0,     
    attribute_paths=attribute_paths,
    units=viz_units,
    opts = dict(ts_opts, xaxis = None, height=120, width=600),
)
hydrograph_bind = pn.bind(
    du.build_hydrograph_from_query_selected_point,
    index=point_selection.param.index,
    points_dmap = points_dmap,          
    scenario = streamflow_scenario,
    reference_time_single=reference_time_player.param.value,  #.value
    value_min=0,     
    attribute_paths=attribute_paths,
    units=viz_units,
    opts = dict(ts_opts, height=220, width=600),
)

###### Apply styles 

tiles_background.opts(**map_opts)
points_dmap.opts(**map_opts, tools=['hover','tap'], colorbar=True, size=5, 
                 toolbar='above', selection_line_width=5, nonselection_line_width=0, nonselection_alpha=0.5)

###### Panel header

header = pn.Row(
            pn.pane.PNG('https://ciroh.ua.edu/wp-content/uploads/2022/08/CIROHLogo_200x200.png', width=60),
            pn.pane.HTML(f"CIROH Tools for Exploratory Evaluation in Hydrology Research (TEEHR)----Example 1: Forecast Data Exploration<br> - {scenario_text}", 
                         sizing_mode="stretch_width", style={'font-size': '18px', 'font-weight': 'bold'}),
)
# Build the Panel layout
layout = \
    pn.Column(
        pn.Column(pn.Spacer(height=10), header, width=1100),
        pn.Row(
            pn.Column(pn.Spacer(height=20), metric_selector, width=200),
            pn.Column(pn.Spacer(height=20), huc2_selector, width=200),
            pn.Spacer(width=40),
            pn.Column(current_ref_time, reference_time_player),
            pn.Spacer(width=40), timeseries_legend,
        ),
        pn.Row(
            tiles_background*points_dmap*gage_basin_dmap, 
            pn.Column(hyetograph_bind, hydrograph_bind),
         )
)
# launch the layout
layout